In [2]:
import tensorflow as tf
from tensorflow import keras
import os


In [3]:
_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin=_URL,fname = 'flower_photos.tgz',extract=True,cache_subdir='/content')
base_dir = os.path.join(os.path.dirname(zip_file),'flower_photos')


228827136/228813984 [==============================] - 2s 0us/step


In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./225,
    validation_split=0.2
)
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,
                 IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
    )

val_generator = datagen.flow_from_directory (
     base_dir,
    target_size=(IMAGE_SIZE,
                 IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
  f.write(labels)



{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [6]:
IMG_SHAPE =(IMAGE_SIZE,IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [7]:
base_model.trainable = False

In [8]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Conv2D(32,3,activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(5,activation='softmax')
])

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
epochs = 10
history = model.fit(
    train_generator,
    epochs = epochs, 
    validation_data = val_generator)

Epoch 1/10
46/46 [==============================] - 129s 3s/step - loss: 0.7999 - accuracy: 0.7475 - val_loss: 0.4413 - val_accuracy: 0.8509
Epoch 2/10
46/46 [==============================] - 122s 3s/step - loss: 0.2846 - accuracy: 0.9034 - val_loss: 0.4757 - val_accuracy: 0.8358
Epoch 3/10
46/46 [==============================] - 123s 3s/step - loss: 0.1945 - accuracy: 0.9354 - val_loss: 0.4288 - val_accuracy: 0.8646
Epoch 4/10
46/46 [==============================] - 120s 3s/step - loss: 0.1124 - accuracy: 0.9636 - val_loss: 0.4464 - val_accuracy: 0.8741
Epoch 5/10
46/46 [==============================] - 123s 3s/step - loss: 0.0772 - accuracy: 0.9806 - val_loss: 0.4620 - val_accuracy: 0.8646
Epoch 6/10
46/46 [==============================] - 140s 3s/step - loss: 0.0552 - accuracy: 0.9847 - val_loss: 0.4742 - val_accuracy: 0.8728
Epoch 7/10
46/46 [==============================] - 139s 3s/step - loss: 0.0290 - accuracy: 0.9966 - val_loss: 0.4840 - val_accuracy: 0.8728
Epoch 8/10
46

In [14]:
saved_model_dir = ''
tf.saved_model.save(model,saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: /tmp/tmpmv5xywoa/assets


INFO:tensorflow:Assets written to: /tmp/tmpmv5xywoa/assets


In [16]:
from google.colab import files
files.download('model.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>